## Pipeline обработки запроса

### Загрузка данных и функций (достаточно выполнить 1 раз)

In [ ]:
from main import setup_environment, load_data
from retriever import retriever
from planner import planner
from grounder import grounder
from executor import executor
from openai import OpenAI
from config import PipelineConfig


api_key, db_path = setup_environment()
db = load_data(db_path, wave_filter=["2025-03"])

client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=api_key,
)

### Настройка используемых моделей и их температур

In [ ]:
PPL_cfg = PipelineConfig.setup(
    df=db, client=client,
    # параметры ретривера
    retriever_params={
        # "model": "alibaba/tongyi-deepresearch-30b-a3b:free",
        "model": "deepseek/deepseek-chat-v3.1",
        "temperature": 0.4
    },
    # параметры планировщика
    planner_params={
        "model": "meta-llama/llama-3.3-70b-instruct:free",
        "temperature": 0.1
    }
)

### Еще варианты моделей
# "alibaba/tongyi-deepresearch-30b-a3b:free"
# "meta-llama/llama-4-maverick:free"
# !!! "deepseek/deepseek-chat-v3.1" # Платная модель

### Retriever

Извлекает релевантные вопросы на основе запроса пользователя и всего набора вопросов

Пользовательский запрос

In [ ]:
user_query = "Я хочу оценить распределение потребительских предпочтений по сетям магазинов в Москве"

Обращение к LLM

> ! Возможны проблемы парсинга ответов

In [ ]:
retriever_out = retriever(user_query, PPL_cfg)

In [ ]:
print(retriever_out)

### Planner

Строит план на основе пользовательского запроса и набора релевантных ответов, к которым подмешиваются их ответы

В план включаются команды из `capability_spec.py`

In [ ]:
planner_out = planner(user_query, retriever_out, PPL_cfg)

In [ ]:
print(planner_out)

### Grounder

Привязка шагов плана к имеющимся функциям (из `operations.py`)

In [ ]:
grounder_out = grounder(planner_out, PPL_cfg)

In [ ]:
print(grounder_out)

### Executor

Валидирует план, осуществляет топологическую сортировку, выполняет последовательность шагов

In [ ]:
ctx = {"dataset": db}
ctx = executor(grounder_out, ctx)